##### What is this notebook about?
- This notebook shows how to implement a simple dataloader for LLM pretraining in pytorch:
    - The dataloader iterates over the input dataset and returns model inputs and targets
    - LLM models predict/generate the next word in a sequence, so the next word in the sequence is the model target.
    - We divide the dataset into chunks. For each text chunk, we want the inputs and targets. The targets are the inputs shifted by one position to the right



In [1]:
import torch
import tiktoken
from torch.utils.data import Dataset, DataLoader

### Dataset & EDA

In [2]:
# Sample data for this notebook
with open("./data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
# Perform some EDA (exploratory data analysis)
# Tokenize the input - check encode & decode

tokenizer = tiktoken.get_encoding("gpt2")
token_ids = tokenizer.encode(raw_text, allowed_special={"<|endoftext|>"})
print(f"Number of tokens: {len(token_ids)}")
print("First 20 tokens", token_ids[:20])

text_out = tokenizer.decode(token_ids)
print("First 20 characters - detokenized", text_out[:20])
print("First 20 characters - original",raw_text[:20])

print("Vocab size", tokenizer.n_vocab)
print("special token", tokenizer.eot_token)


Number of tokens: 5145
First 20 tokens [40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438]
First 20 characters - detokenized I HAD always thought
First 20 characters - original I HAD always thought
Vocab size 50257
special token 50256


In [4]:
# Get a sample
sample = token_ids[50:150]
len(sample)

# Show the model inputs & target for this sample
context_size = 4
x = sample[:context_size]
y = sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

# Show input tokens & target tokens
for i in range(1, context_size+1):
    input = sample[:i]
    target = sample[i]
    print(input, "->", target)

# Show input text & target text
for i in range(1, context_size+1):
    input = sample[:i]
    target = sample[i]
    print(tokenizer.decode(input), "->", tokenizer.decode([target]))

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]
[290] -> 4920
[290, 4920] -> 2241
[290, 4920, 2241] -> 287
[290, 4920, 2241, 287] -> 257
 and ->  established
 and established ->  himself
 and established himself ->  in
 and established himself in ->  a


### Create Custom Dataloader 
- Create custom dataset (inherited from pytorch `Dataset`) - with `__init__()`, `__len__()`, `___getitem__` functions.
- Optionally - Can use custom `collate()` function to convert batch of examples into desired batch of pytorch tensors that will be used in training / inference
- Create dataloader with this custom dataset (and optional `collate` function) using pytorch `DataLoader` 

In [5]:
class CustomDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Vocab size
        self.vocab_size = tokenizer.n_vocab

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [6]:
batch_size = 8
max_length = 4
stride=max_length
num_workers = 0

# Initialize the tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

# Create dataset
dataset = CustomDataset(raw_text, tokenizer, max_length, stride)

# Create dataloader 
# (shuffle set to False just for demo. Should be True for training dataloader)
dataloader = DataLoader(
    dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=num_workers)

In [7]:
# Check dataloader
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(inputs)
print(targets)

# # Same as:
# dataloader = DataLoader(
#     dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=num_workers)
# for inputs, targets in dataloader:
#     print(inputs)
#     print(targets)
#     break

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [8]:
# Dataloader for LLM 
# - Convert input tokens into embeddings
# - Optionally add positional embeddings here:
#   - Looks at LLM components notebook/code for different types of positional embeddings

vocab_size = dataset.vocab_size
emb_dim = 256
context_length = 1024

token_embedding_layer = torch.nn.Embedding(vocab_size, emb_dim)
pos_embedding_layer = torch.nn.Embedding(context_length, emb_dim)

for batch in dataloader:
    x, y = batch

    token_embeddings = token_embedding_layer(x)  # bs, max_length, emb_dim
    pos_embeddings = pos_embedding_layer(torch.arange(max_length)) # bs, max_length, emb_dim 

    # input embeddings into LLM
    input_embeddings = token_embeddings + pos_embeddings

    break

In [9]:
print(input_embeddings.shape) # bs, max_length, emb_dim

torch.Size([8, 4, 256])


### References:

> https://github.com/rasbt/LLMs-from-scratch

